In [59]:
import numpy as np
import pandas as pd
from spacy.lang.fr.stop_words import STOP_WORDS

In [60]:
print("bonjour")

bonjour


In [61]:
df = pd.read_csv("data/Keyword_with_PKE_queneau_frprotocol_V2.csv")
df.head()

,Unnamed: 0,title,texte,texte_keyword,keyword_without_ent,keyword_without_stop_word,keywords_lematisations
0,0,Notations.,"Dans l'S, à une heure d'affluence. Un type dan...",type$ans$chapeau$cordon$question$ruban$affluen...,type$ans$chapeau$cordon$question$ruban$affluen...,type$ans$chapeau$cordon$question$ruban$affluen...,heure$affluence$type$an$chapeau$cordon$ruban$c...
1,1,En partie double.,"Vers le milieu de la journée et à midi, je me ...",ligne s$transports$place$sort$fois qu'on$minut...,s$transports$place$sort$fois$qu'on$minutes$rev...,ligne$s$transports$place$sort$fois$qu'on$minut...,terrasse$arrière$véhicule$transport$ligne$fois...
2,2,Litotes.,Nous étions quelques-uns à nous déplacer de co...,jeune homme$air$instants$compagnie$camarade$mo...,jeune$homme$air$instants$compagnie$camarade$mo...,jeune$homme$air$instants$compagnie$camarade$mo...,jeune$homme$air$instant$Monsieur$côté$heure$co...
3,3,Métaphoriquement.,"Au centre du jour, jeté dans le tas des sardin...",grosse carapace$grand cou$morne désert$jour$ai...,grosse$carapace$grand$cou$morne$désert$jour$ai...,grosse$carapace$grand$cou$morne$désert$jour$ai...,jour$tas$sardine$gros$carapace$poulet$grand$co...
4,4,Rétrograde.,"Tu devrais ajouter un bouton à ton pardessus, ...",jeune homme$plate-forme$chapeau$place$midi$por...,jeune$homme$plate-forme$chapeau$place$midi$por...,jeune$homme$plate-forme$chapeau$place$midi$por...,avidité$place$fois$jeune$homme$porteur$chapeau...


In [62]:
df=df.dropna()
df=df.reset_index(drop=True)

In [63]:
type(df)

pandas.core.frame.DataFrame

In [64]:
keys =[key for key in df.keys() if "keyword" in key]
dict_matrice ={}
for key in keys:
    keywords = "$".join(df[key].tolist())

    symbols = "!\"#%&()*+-./:;<=>?@[\]^_`{|}~\n,."
    for i in symbols:
        keywords.replace(i," ")

    list_keywords=keywords.split('$')
    list_keywords=list(set(list_keywords))
    #list_keywords = [word for word in list_keywords if len(word)>1]
    list_keywords = list(set(list_keywords))
    dict_matrice[key]=list_keywords


for key in keys :
    print(len(dict_matrice[key]))

675
671
766
679


In [65]:
def mytokenizer(text):
    return text.split()
from sklearn.feature_extraction.text import CountVectorizer
corpus=df["texte"]
dic_vectors = {}
for key in keys:
    vocab = dict_matrice[key]
    vectorizer = CountVectorizer(vocabulary=vocab, tokenizer = mytokenizer)
    X = vectorizer.fit_transform(corpus)
    vectorizer.get_feature_names()
    dic_vectors[key] =  X.toarray()



c:\users\le muletier\pycharmprojects\test-de-quenaud\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\le muletier\pycharmprojects\test-de-quenaud\venv\lib\site-packages\sklearn\feature_extraction\text.py:1323: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  "Upper case characters found in"


In [66]:
from numpy.linalg import norm

In [67]:

df_corélations_cos= pd.DataFrame()


for key in keys:
    for index , vector in enumerate(dic_vectors[key]):
        df_corélations_cos.loc[index,"title"]=df.loc[index,"title"]
        df_corélations_cos.loc[index, "PKE_corrélation_"+str(key)] = np.dot(dic_vectors[key][0],vector)/(norm(dic_vectors[key][0])*norm(vector))



In [68]:
df_corélations_cos.head()

,title,PKE_corrélation_texte_keyword,PKE_corrélation_keyword_without_ent,PKE_corrélation_keyword_without_stop_word,PKE_corrélation_keywords_lematisations
0,Notations.,1.000000,1.000000,1.000000,1.000000
1,En partie double.,0.202031,0.202777,0.183340,0.301232
2,Litotes.,0.000000,0.000000,0.000000,0.172133
3,Métaphoriquement.,0.094491,0.066815,0.074125,0.086066
4,Rétrograde.,0.296500,0.322749,0.296500,0.298142


In [69]:
df_corélations_cos.to_csv("results/result_pke_correlation.csv")